# 2023IKCEST第五届“一带一路”国际大数据竞赛
# 一、背景介绍

本届大数据竞赛在中国工程院、教育部高等学校大学计算机课程教学指导委员会及丝绸之路大学联盟的指导下由联合国教科文组织国际工程科技知识中心（IKCEST）、中国工程科技知识中心（CKCEST）、百度公司及西安交通大学共同主办，旨在放眼“一带一路”倡议沿线国家，通过竞赛方式挖掘全球大数据人工智能尖端人才，实现政府—产业—高校合力推动大数据产业研究、应用、发展的目标，进一步夯实赛事的理论基础与实践基础，加快拔尖AI创新人才培养。

# 二、赛题介绍
随着新媒体时代信息媒介的多元化发展，各种内容大量活跃在媒体内中，与此同时各类虚假信息也充斥着社交媒体，影响着公众的判断和决策。如何在大量的文本、图像等多模态信息中，通过大数据与人工智能技术，纠正和消除虚假错误信息，对于网络舆情及社会治理有着重大意义。

本次赛题要求选手基于官方指定数据集，通过建模同一事实跨模态数据之间的关系 （主要是文本和图像），实现对任一模态信息能够进行虚假和真实性的检测。鼓励参赛选手通过大模型解决问题，进行技术探索。

In [1]:
#环境安装
!pip install paddlenlp==2.4.2

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip available: 22.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip


# 1.解压套件安装PaddleOCR套件
![](https://ai-studio-static-online.cdn.bcebos.com/9e1e197487d1466fa662471a62f610b6357dad3b73ea4de698b59a020b4fed07)


In [2]:
%cd PaddleOCR-2.6.0
# 安装依赖库
!pip install -r requirements.txt -i https://mirror.baidu.com/pypi/simple
!pip install scikit-image -i https://mirror.baidu.com/pypi/simple
!pip install imgaug -i https://mirror.baidu.com/pypi/simple
!mkdir inference && cd PaddleOCR-2.6.0/inference
# 下载8.6M中文模型的检测模型并解压
! cd inference && wget https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_det_infer.tar && tar xf ch_PP-OCRv3_det_infer.tar
# 下载8.6M中文模型的识别模型并解压
! cd inference && wget https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_rec_infer.tar && tar xf ch_PP-OCRv3_rec_infer.tar
%cd ..

/home/aistudio/data/data244320/PaddleOCR-2.6.0
Looking in indexes: https://mirror.baidu.com/pypi/simple, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip available: 22.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://mirror.baidu.com/pypi/simple, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip available: 22.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://mirror.baidu.com/pypi/simple, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip available: 22.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip
mkdir: 无法创建目录"inference": 文件已存在
--2023-10-20 23:55:43--  https://paddleocr.bj.bcebos.com/PP-OCRv3/chinese/ch_PP-OCRv3_det_infer.tar
正在解析主机 paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 182.61.200.195

# 三、数据集介绍
本次比赛提供从国内外主流社交媒体平台上爬取的含有不同领域声明的数据集。

初赛：训练集与验证集： 提供中文训练集5694条以及英文数据4893条，同时公开英文验证集611条与中文验证集711条供选手优化模型。

初赛评测数据： 提供文娱、经济、健康领域的测试数据，这些领域的数据较容易区分。英文与中文数据集的测试集各600条。参赛队伍上传的结果文本的每一行就是对应的分类结果，该数据不公布，用于评测。


| 0 | 1 | 2 |
| -------- | -------- | -------- |
| non-rumor | rumor  | unverified |



[复赛数据后续见官网通知](https://aistudio.baidu.com/aistudio/competition/detail/1030/0/task-definition)

# 四、数据预处理
**数据集过大，右键选择解压/home/aistudio/data/data229919/data.zip数据集，耐心等待30分钟，直到出现以下文件夹和文件,解压之后硬盘达到约80g（压缩包27g、解压文件之后50g，可以将项目挂载的数据集取消，空余出27g）**
* test
* train
* val
* dataset_items_test.json
* dataset_items_train.json
* dataset_items_val.json

此处将数据集已经放置在queries_dataset_merge文件夹

# 4.1 导包

In [3]:
from functools import partial
import time
import os 
import copy
import json
import gc
from urllib.parse import urlparse
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tqdm import tqdm 
from PIL import Image
import imghdr
import logging
import sys
import cv2

import paddle
from paddlenlp.datasets import load_dataset
import paddle.nn.functional as F
import paddle.nn as nn
import paddlenlp as ppnlp
from paddlenlp.transformers import LinearDecayWithWarmup
from paddle.vision import transforms as T
from paddle.io import Dataset
from paddle.io import DataLoader
from paddle.vision import models
from paddlenlp.transformers import ErnieMModel,ErnieMTokenizer
from paddle.metric import Metric
os.environ["FLAGS_eager_delete_tensor_gb"] = "0.0"
sys.path.insert(0, "PaddleOCR-2.6.0")
import tools.infer.utility as utility
import tools.infer.predict_rec as predict_rec
import tools.infer.predict_det as predict_det
import tools.infer.predict_cls as predict_cls
from ppocr.utils.utility import get_image_file_list, check_and_read
from ppocr.utils.logging import get_logger
from tools.infer.utility import draw_ocr_box_txt, get_rotate_crop_image
seed = 2021
paddle.seed(seed)
np.random.seed(seed)
random.seed(seed)

In [4]:
#读取数据
path = "/home/aistudio/queries_dataset_merge"
data_items_train = json.load(open(os.path.join(path,"dataset_items_train.json")))
data_items_val = json.load(open(os.path.join(path,"dataset_items_val.json")))
data_items_test = json.load(open(os.path.join(path,"dataset_items_val_60.json")))
train_list = glob.glob(os.path.join(path,"train/img/*jpg"))
test_list = glob.glob(os.path.join(path,"test/img/*jpg"))
val_list = glob.glob(os.path.join(path,"val/img/*jpg"))

# 4.2 ocr识别训练集、验证集、测试集中的img图片¶
### 定义ocr生成图片文本的txt格式,代码修改与PaddleOCR-2.6.0/tools/infer/predict_system.py

In [5]:
# #20小时
# 定义函数，用于返回图片的文本信息
!cp predict_system.py   PaddleOCR-2.6.0/tools/infer/
def orc_image(image_pth):
    cmd = f"cd PaddleOCR-2.6.0 && python3 tools/infer/predict_system.py  --det_model_dir=inference/ch_PP-OCRv3_det_infer/ --rec_model_dir=inference/ch_PP-OCRv3_rec_infer/  --image_dir={image_pth} --draw_img_save_dir=ch_PP-OCRv2_results"
    os.system(cmd)
    os.system(f"mv PaddleOCR-2.6.0/ch_PP-OCRv2_results/system_results.txt {image_pth}.txt")
#ocr_image_text = orc_image("/home/aistudio/queries_dataset_merge/val/img/1044.jpg")
# print(ocr_image_text)
for image_path in train_list:
    if not os.path.isfile(image_path+".txt"):
        orc_image(image_path)
for image_path in test_list:
    if not os.path.isfile(image_path+".txt"):
        orc_image(image_path)
for image_path in val_list:
    if not os.path.isfile(image_path+".txt"):
        print(image_path)
        orc_image(image_path)

# 定义频域特征

In [6]:
transform_dct = T.Compose([
                    T.Resize((224,224)),
                    T.ToTensor()
            ])

def process_dct_img(img):
    img = img.numpy() #size = [1, 224, 224]
    height = img.shape[1]
    width = img.shape[2]
    #print('height:{}'.format(height))
    N = 8 
    step = int(height/N) #28

    dct_img = np.zeros((1, N*N, step*step, 1), dtype=np.float32) #[1,64,784,1]
    fft_img = np.zeros((1, N*N, step*step, 1))
    #print('dct_img:{}'.format(dct_img.shape))
    
    i = 0
    for row in np.arange(0, height, step):
        for col in np.arange(0, width, step):
            block = np.array(img[:, row:(row+step), col:(col+step)], dtype=np.float32)
            #print('block:{}'.format(block.shape))
            block1 = block.reshape(-1, step*step, 1) #[batch_size,784,1]
            dct_img[:, i,:,:] = dct(block1) #[batch_size, 64, 784, 1]

            i += 1

    #for i in range(64):
    fft_img[:,:,:,:] = fft(dct_img[:,:,:,:]).real #[batch_size,64, 784,1]
    
    fft_img = paddle.to_tensor(fft_img).astype('float32')#.float() #[batch_size, 64, 784, 1]
    new_img = F.interpolate(fft_img, size=[250,1]) #[batch_size, 64, 250, 1]
    new_img = new_img.squeeze(0).squeeze(-1) #torch.size = [64, 250]
    
    return new_img   

class DctStem(nn.Layer):
    def __init__(self, kernel_sizes, num_channels):
        super(DctStem, self).__init__()
        self.convs = nn.Sequential(
            ConvBNRelu2d(in_channels=1,
                         out_channels=num_channels[0],
                         kernel_size=kernel_sizes[0]),
            ConvBNRelu2d(
                in_channels=num_channels[0],
                out_channels=num_channels[1],
                kernel_size=kernel_sizes[1],
            ),
            ConvBNRelu2d(
                in_channels=num_channels[1],
                out_channels=num_channels[2],
                kernel_size=kernel_sizes[2],
            ),
            nn.MaxPool2D((1, 2)),
        )

    def forward(self, dct_img):
        x = dct_img.unsqueeze(1)
        x = x.unsqueeze(1)
        img = self.convs(x)
        img = img.transpose([0, 2, 1, 3])

        return img

class DctInceptionBlock(nn.Layer):
    def __init__(
        self,
        in_channel=128,
        branch1_channels=[64],
        branch2_channels=[48, 64],
        branch3_channels=[64, 96, 96],
        branch4_channels=[32],
    ):
        super(DctInceptionBlock, self).__init__()

        self.branch1 = ConvBNRelu2d(in_channels=in_channel,
                                    out_channels=branch1_channels[0],
                                    kernel_size=1)

        self.branch2 = nn.Sequential(
            ConvBNRelu2d(in_channels=in_channel,
                         out_channels=branch2_channels[0],
                         kernel_size=1),
            ConvBNRelu2d(
                in_channels=branch2_channels[0],
                out_channels=branch2_channels[1],
                kernel_size=3,
                padding=(0, 1),
            ),
        )

        self.branch3 = nn.Sequential(
            ConvBNRelu2d(in_channels=in_channel,
                         out_channels=branch3_channels[0],
                         kernel_size=1),
            ConvBNRelu2d(
                in_channels=branch3_channels[0],
                out_channels=branch3_channels[1],
                kernel_size=3,
                padding=(0, 1),
            ),
            ConvBNRelu2d(
                in_channels=branch3_channels[1],
                out_channels=branch3_channels[2],
                kernel_size=3,
                padding=(0, 1),
            ),
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2D(kernel_size=(1, 3), stride=1, padding=(0, 1)),
            ConvBNRelu2d(in_channels=in_channel,
                         out_channels=branch4_channels[0],
                         kernel_size=1),
        )

    def forward(self, x):

        x = x.transpose([0, 2, 1, 3])
        # y = x
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        out4 = self.branch4(x)
        out = paddle.concat(x = [out1, out2, out3, out4], axis=1)
        out = out.transpose([0, 2, 1, 3])

        return out
def ConvBNRelu2d(in_channels, out_channels, kernel_size, stride=1, padding=0):
    return nn.Sequential(
        nn.Conv2D(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(1, kernel_size),
            stride=stride,
            padding=padding,
        ),
        nn.BatchNorm2D(out_channels),
        nn.ReLU(name=True),
    )



class DctCNN(nn.Layer):
    def __init__(self,
                 model_dim,
                 dropout,
                 kernel_sizes,
                 num_channels,
                 in_channel=128,
                 branch1_channels=[64],
                 branch2_channels=[48, 64],
                 branch3_channels=[64, 96, 96],
                 branch4_channels=[32],
                 out_channels=64):

        super(DctCNN, self).__init__()

        self.stem = DctStem(kernel_sizes, num_channels)

        self.InceptionBlock = DctInceptionBlock(
            in_channel,
            branch1_channels,
            branch2_channels,
            branch3_channels,
            branch4_channels,
        )

        self.maxPool = nn.MaxPool2D((1, 122))

        self.dropout = nn.Dropout(dropout)

        self.conv = ConvBNRelu2d(branch1_channels[-1] + branch2_channels[-1] +
                               branch3_channels[-1] + branch4_channels[-1],
                               out_channels,
                               kernel_size=1)

    def forward(self, dct_img):
        dct_f = self.stem(dct_img)
        x = self.InceptionBlock(dct_f)
        x = self.maxPool(x)
        x = x.transpose([0, 2, 1, 3])
        x = self.conv(x)
        x = x.transpose([0, 2, 1, 3])
        x = x.squeeze(-1)
        x = x.reshape([-1,4096])

        return x
    
class NetShareFusion(nn.Layer):
    def __init__(self,
                 kernel_sizes,
                 num_channels,
                 model_dim,
                 drop_and_BN,
                 num_labels=2,
                 dropout=0.5):

        super(NetShareFusion, self).__init__()

        self.model_dim = model_dim
        self.drop_and_BN = drop_and_BN
        self.dropout = nn.Dropout(dropout)

        #dct_image
        self.dct_img = DctCNN(model_dim,
                              dropout,
                              kernel_sizes,
                              num_channels,
                              in_channel=128,
                              branch1_channels=[64],
                              branch2_channels=[48, 64],
                              branch3_channels=[64, 96, 96],
                              branch4_channels=[32],
                              out_channels=64)
        self.linear_dct = nn.Linear(4096, model_dim)
        self.bn_dct = nn.BatchNorm1D(model_dim)

       
        #classifier
        self.linear1 = nn.Linear(model_dim, 35)
        self.bn_1 = nn.BatchNorm1D(35)
        self.linear2 = nn.Linear(35, num_labels)
        self.softmax = nn.Softmax(axis=1)
    
    def drop_BN_layer(self, x, part='dct'):
        if part == 'dct':
            bn = self.bn_dct

        if self.drop_and_BN == 'drop-BN':
            x = self.dropout(x)
            x = bn(x)
        
        return x

    def forward(self,dct_img):

        #dct_feature
        dct_out = self.dct_img(dct_img)
        dct_out = F.relu(self.linear_dct(dct_out))
        dct_out = self.drop_BN_layer(dct_out, part='dct')

        return dct_out

读取数据中的每一个样本：图像img、文本caption、对应的img_html_news、inverse_search为支持图像img和文本caption的证据材料

In [7]:
def process_string(input_str):
    input_str = input_str.replace('&#39;', ' ')
    input_str = input_str.replace('<b>','')
    input_str = input_str.replace('</b>','')
    #input_str = unidecode(input_str)  
    return input_str
    
class NewsContextDatasetEmbs(Dataset):
    def __init__(self, context_data_items_dict, queries_root_dir, split):
        self.context_data_items_dict = context_data_items_dict
        self.queries_root_dir = queries_root_dir
        self.idx_to_keys = list(context_data_items_dict.keys())
        self.transform =T.Compose([
                        T.Resize(256),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ])
        self.split=split
    def __len__(self):
        return len(self.context_data_items_dict)   


    def load_img_pil(self,image_path):
        if imghdr.what(image_path) == 'gif': 
            try:
                with open(image_path, 'rb') as f:
                    img = Image.open(f)
                    return img.convert('RGB')
            except:
                return None 
        with open(image_path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
    def load_imgs_direct_search(self,item_folder_path,direct_dict):   
        list_imgs_tensors = []
        count = 0   
        keys_to_check = ['images_with_captions','images_with_no_captions','images_with_caption_matched_tags']
        for key1 in keys_to_check:
            if key1 in direct_dict.keys():
                for page in direct_dict[key1]:
                    image_path = os.path.join(item_folder_path,page['image_path'].split('/')[-1])
                    try:
                        pil_img = self.load_img_pil(image_path)
                    except Exception as e:
                        print(e)
                        print(image_path)
                    if pil_img == None: continue
                    transform_img = self.transform(pil_img)
                    count = count + 1 
                    list_imgs_tensors.append(transform_img)
        stacked_tensors = paddle.stack(list_imgs_tensors, axis=0)
        return stacked_tensors

    def load_captions(self,inv_dict):
        captions = ['']
        pages_with_captions_keys = ['all_fully_matched_captions','all_partially_matched_captions']
        for key1 in pages_with_captions_keys:
            if key1 in inv_dict.keys():
                for page in inv_dict[key1]:
                    if 'title' in page.keys():
                        item = page['title']
                        item = process_string(item)
                        captions.append(item)
                    
                    if 'caption' in page.keys():
                        sub_captions_list = []
                        unfiltered_captions = []
                        for key2 in page['caption']:
                            sub_caption = page['caption'][key2]
                            sub_caption_filter = process_string(sub_caption)
                            if sub_caption in unfiltered_captions: continue 
                            sub_captions_list.append(sub_caption_filter) 
                            unfiltered_captions.append(sub_caption) 
                        captions = captions + sub_captions_list 
                    
        pages_with_title_only_keys = ['partially_matched_no_text','fully_matched_no_text']
        for key1 in pages_with_title_only_keys:
            if key1 in inv_dict.keys():
                for page in inv_dict[key1]:
                    if 'title' in page.keys():
                        title = process_string(page['title'])
                        captions.append(title)
        return captions

    def load_captions_weibo(self,direct_dict):
        captions = ['']
        keys = ['images_with_captions','images_with_no_captions','images_with_caption_matched_tags']
        for key1 in keys:
            if key1 in direct_dict.keys():
                for page in direct_dict[key1]:
                    if 'page_title' in page.keys():
                        item = page['page_title']
                        item = process_string(item)
                        captions.append(item)
                    if 'caption' in page.keys():
                        sub_captions_list = []
                        unfiltered_captions = []
                        for key2 in page['caption']:
                            sub_caption = page['caption'][key2]
                            sub_caption_filter = process_string(sub_caption)
                            if sub_caption in unfiltered_captions: continue 
                            sub_captions_list.append(sub_caption_filter) 
                            unfiltered_captions.append(sub_caption) 
                        captions = captions + sub_captions_list 
        #print(captions)
        return captions
        #加载img文件夹
    def load_queries(self,key):
        caption = self.context_data_items_dict[key]['caption']
        image_path = os.path.join(self.queries_root_dir,self.context_data_items_dict[key]['image_path'])
        pil_img = self.load_img_pil(image_path)
        # ======================
        with open(image_path+".txt") as f:
            data =f.readlines()
        text_all = []
        if len(data) !=0:
            for i in eval(data[0].split("\t")[1]):
                text_all.append(i['transcription'])
            ocr_image_text = [",".join(text_all)]
        else:
            ocr_image_text = [""]
        # ======================
        dct_img = transform_dct(Image.open(image_path).convert('L'))
        #ocr_image_text = [orc_image(image_path)]
        transform_img = self.transform(pil_img)
        return transform_img, caption,ocr_image_text,dct_img
    def __getitem__(self, idx):
        #print(idx)
        #print(self.context_data_items_dict)      
        #idx = idx.tolist()               
        key = self.idx_to_keys[idx]
        #print(key)
        item=self.context_data_items_dict.get(str(key))
        #print(item)
        # 如果为test没有label属性
        #print(self.split)
        if self.split=='train' or self.split=='val':
            label = paddle.to_tensor(int(item['label']))
            direct_path_item = os.path.join(self.queries_root_dir,item['direct_path'])
            inverse_path_item = os.path.join(self.queries_root_dir,item['inv_path'])
            inv_ann_dict = json.load(open(os.path.join(inverse_path_item, 'inverse_annotation.json')))
            direct_dict = json.load(open(os.path.join(direct_path_item, 'direct_annotation.json')))
            captions= self.load_captions(inv_ann_dict)
            captions += self.load_captions_weibo(direct_dict)
            imgs = self.load_imgs_direct_search(direct_path_item,direct_dict)  
            #imgs_text = self.load_imgs_direct_search_text(direct_path_item,direct_dict)  
            qImg,qCap,imgs_text,dct_img =  self.load_queries(key)
            sample = {'label': label, 'caption': captions,'imgs': imgs,  'qImg': qImg, 'qCap': qCap,"imgs_text":imgs_text,"dct_img":dct_img}
        else:
            direct_path_item = os.path.join(self.queries_root_dir,item['direct_path'])
            inverse_path_item = os.path.join(self.queries_root_dir,item['inv_path'])
            inv_ann_dict = json.load(open(os.path.join(inverse_path_item, 'inverse_annotation.json')))
            direct_dict = json.load(open(os.path.join(direct_path_item, 'direct_annotation.json')))
            captions= self.load_captions(inv_ann_dict)
            captions += self.load_captions_weibo(direct_dict)
            imgs = self.load_imgs_direct_search(direct_path_item,direct_dict)    
            #imgs_text = self.load_imgs_direct_search_text(direct_path_item,direct_dict) 
            qImg,qCap,imgs_text,dct_img =  self.load_queries(key)
            sample = {'caption': captions,'imgs': imgs,  'qImg': qImg, 'qCap': qCap,"imgs_text":imgs_text,"dct_img":dct_img}
        #print(sample)
        #print(len(captions)) 
        #print(type(imgs))
        #print(imgs.size)
        #print(imgs.shape)  
        return sample,  len(captions), imgs.shape[0]


In [8]:
#### load Datasets ####
train_dataset = NewsContextDatasetEmbs(data_items_train, path,'train')
val_dataset = NewsContextDatasetEmbs(data_items_val,path,'val')
test_dataset = NewsContextDatasetEmbs(data_items_test,path,'test')

In [9]:
def collate_context_bert_train(batch):
    #print(batch)
    samples = [item[0] for item in batch]
    max_captions_len = max([item[1] for item in batch])
    max_images_len = max([item[2] for item in batch])
    qCap_batch = []
    qImg_batch = []
    img_batch = []
    img_text_batch = []
    dct_img_batch = []
    cap_batch = []
    labels = [] 
    for j in range(0,len(samples)):  
        sample = samples[j]    
        labels.append(sample['label'])
        captions = sample['caption']
        cap_len = len(captions)
        for i in range(0,max_captions_len-cap_len):
            captions.append("")
        if len(sample['imgs'].shape) > 2:
            padding_size = (max_images_len-sample['imgs'].shape[0], sample['imgs'].shape[1], sample['imgs'].shape[2], sample['imgs'].shape[3])
        else:
            padding_size = (max_images_len-sample['imgs'].shape[0],sample['imgs'].shape[1])
        padded_mem_img = paddle.concat((sample['imgs'], paddle.zeros(padding_size)),axis=0)
        #print(1)
        img_batch.append(padded_mem_img)#pad证据图片
        cap_batch.append(captions)
        img_text_batch.append(sample['imgs_text'])
        dct_img_batch.append(sample['dct_img'])
        qImg_batch.append(sample['qImg'])#[3, 224, 224]
        qCap_batch.append(sample['qCap'])     
    #print(labels)   
    #print(img_batch)
    img_batch = paddle.stack(img_batch, axis=0)
    qImg_batch = paddle.stack(qImg_batch, axis=0)
    labels = paddle.stack(labels, axis=0) 
    #print(3)  
    return labels, cap_batch, img_batch, qCap_batch, qImg_batch,img_text_batch,dct_img_batch

def collate_context_bert_test(batch):
    samples = [item[0] for item in batch]
    max_captions_len = max([item[1] for item in batch])
    max_images_len = max([item[2] for item in batch])
    qCap_batch = []
    qImg_batch = []
    img_batch = []
    img_text_batch = []
    dct_img_batch = []
    cap_batch = []
    for j in range(0,len(samples)):  
        sample = samples[j]    
        captions = sample['caption']
        cap_len = len(captions)
        for i in range(0,max_captions_len-cap_len):
            captions.append("")
        if len(sample['imgs'].shape) > 2:
            padding_size = (max_images_len-sample['imgs'].shape[0],sample['imgs'].shape[1],sample['imgs'].shape[2],sample['imgs'].shape[3])
        else:
            padding_size = (max_images_len-sample['imgs'].shape[0],sample['imgs'].shape[1])
        padded_mem_img = paddle.concat((sample['imgs'], paddle.zeros(padding_size)),axis=0)
        img_batch.append(padded_mem_img)
        cap_batch.append(captions)
        img_text_batch.append(sample['imgs_text'])
        dct_img_batch.append(sample['dct_img'])
        qImg_batch.append(sample['qImg'])
        qCap_batch.append(sample['qCap'])        
    img_batch = paddle.stack(img_batch, axis=0)
    qImg_batch = paddle.stack(qImg_batch, axis=0)
    return cap_batch, img_batch, qCap_batch, qImg_batch,img_text_batch,dct_img_batch

In [10]:
# load DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn = collate_context_bert_train, return_list=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn = collate_context_bert_train,  return_list=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn = collate_context_bert_test, return_list=True)

# 四、模型构建
**本次赛题为一个NLP与多模态的分类赛题，整体建模采用特征提取、特征交互、预测分类三个阶段**

**特征提取：** 对于图像数据，使用ResNet模型进行特征提取、对于文本数据，使用预训练模型Ernie-m多语言模型对中文和英文同时处理，qCap,qImg,（需要验证的标题或图像材料）、caps,imgs（支持验证的文本、图像证据材料）

**特征交互**：使用多头自注意力机制，将标题与文本证据材料交互、图像与图像证据材料交互，输出与需要验证的标题和图像的相关证据特征caps_feature、imgs_features

**预测分类：** 最后使用全连接层将标题特征、图像特征、相关的文本证据特征、相关的图像证据特征拼接输入到分类器得到最终结果
![](https://ai-studio-static-online.cdn.bcebos.com/3f29e3f853b9445fbeb24189103cdbbcb8364498dc484593a891839994dadbd6)



## 多语言预训练模型ERNIE-M
2021年，百度发布多语言预训练模型ERNIE-M。ERNIE-M通过对96门语言的学习，使得一个模型能同时理解96种语言，该项技术在5类典型跨语言理解任务上刷新世界最好效果。

## ERNIE-M原理
ERNIE-M基于飞桨PaddlePaddle框架训练，该模型构建了大小为25万的多语言词表，涵盖了96种语言的大多数常见词汇，训练语料包含了汉语、英语、法语、南非语、阿尔巴尼亚语、阿姆哈拉语、梵语、阿拉伯语、亚美尼亚语、阿萨姆语、阿塞拜疆语等96种语言，约1.5万亿字符。

ERNIE-M的学习过程由两阶段组成。第一阶段从少量的双语语料中学习跨语言理解能力，使模型学到初步的语言对齐关系；第二阶段使用回译的思想，通过大量的单语语料学习，增强模型的跨语言理解能力。

[百度NLP知乎介绍](https://zhuanlan.zhihu.com/p/344810337)


![](https://ai-studio-static-online.cdn.bcebos.com/5aad6df2b5b64d5bab0454299249c04e85cc227cf1be4352b09885670a2b7a07)


![](https://ai-studio-static-online.cdn.bcebos.com/baf24e95ce694cac8742e7075be7fcfd843609fc263f406cbece98149144619b)


In [11]:
class EncoderCNN(nn.Layer):
    def __init__(self, resnet_arch = 'resnext101_64x4d'):
        super(EncoderCNN, self).__init__()
        if resnet_arch == 'resnet101':
            resnet = models.resnet101(pretrained=True)
        elif resnet_arch == 'resnext101_64x4d':
            resnet = models.resnext101_64x4d(pretrained=True)
        modules = list(resnet.children())[:-2]
        self.resnet = nn.Sequential(*modules)
        self.adaptive_pool = nn.AdaptiveAvgPool2D((1, 1))
    def forward(self, images, features='pool'):
        out = self.resnet(images)
        if features == 'pool':
            out = self.adaptive_pool(out)
            out = paddle.reshape(out, (out.shape[0],out.shape[1]))
        return out

class NetWork(nn.Layer):
    def __init__(self, mode):
        super(NetWork, self).__init__()
        self.mode = mode           
        self.ernie = ErnieMModel.from_pretrained('ernie-m-base')
        self.tokenizer = ErnieMTokenizer.from_pretrained('ernie-m-base')
        self.resnet = EncoderCNN()
        self.dct_feature_model = NetShareFusion(
                kernel_sizes= [3, 3, 3],
                num_channels=[32, 64, 128],
                model_dim=256,
                dropout=0.5,
                drop_and_BN=['drop-BN'])
        # 只能存在一个
        self.ocr = False
        self.dct = False
        self.fusion_type = True
        if self.dct:
            self.classifier1 = nn.Linear(1*(768+2048)+768*3+256,1024) 
        if self.ocr:
            self.classifier1 = nn.Linear(1*(768+2048)+768*4,1024) 
        self.classifier2 = nn.Linear(1024,3)
        self.attention_text = nn.MultiHeadAttention(768,16)
        self.attention_image = nn.MultiHeadAttention(2048,16)
        self.attention_image_text = nn.MultiHeadAttention(768,4)
        self.mid_attention_image_text = nn.Linear(2048,128*768) 
        self.mlp1 = paddle.nn.Sequential(
                paddle.nn.Linear(2048*2+768*3, 1024),
                paddle.nn.ReLU(),
                paddle.nn.Dropout(0.2),
                paddle.nn.Linear(1024, 3)
            )

        self.mlp2 = paddle.nn.Sequential(
                paddle.nn.Linear(2048*1+768*4, 1024),
                paddle.nn.ReLU(),
                paddle.nn.Dropout(0.2),
                paddle.nn.Linear(1024, 3)
            )

        self.mlp3 = paddle.nn.Sequential(
                paddle.nn.Linear(768, 256),
                paddle.nn.ReLU(),
                paddle.nn.Dropout(0.2),
                paddle.nn.Linear(256, 3)
            )

        self.mlp4 = paddle.nn.Sequential(
                paddle.nn.Linear(2048, 1024),
                paddle.nn.ReLU(),
                paddle.nn.Dropout(0.2),
                paddle.nn.Linear(1024, 3)
            )

        self.fusion_mlp = paddle.nn.Sequential(
                        paddle.nn.Linear(2048, 768),
                        paddle.nn.ReLU(),
                        paddle.nn.Dropout(0.2),
                    )
        self.fusion_mlp2 = paddle.nn.Sequential(
                paddle.nn.Linear(768, 256),
                paddle.nn.ReLU(),
                paddle.nn.Dropout(0.2),
                paddle.nn.Linear(256, 3)
            )
        self.fusion_attention1 = nn.MultiHeadAttention(768,16)
        self.fusion_attention2 = nn.MultiHeadAttention(768,16)
        self.fusion_attention3 = nn.MultiHeadAttention(768,16)
        self.fusion_attention4 = nn.MultiHeadAttention(768,16)
        self.dropout = nn.Dropout(p=0.5)
        if self.mode == 'text':
            self.classifier = nn.Linear(768,3)
        self.resnet.eval()

    def forward(self,qCap,qImg,caps,imgs,img_text,dct_img):
        self.resnet.eval()
        encode_dict_qcap = self.tokenizer(text = qCap ,max_length = 128 ,truncation=True, padding='max_length')
        input_ids_qcap = encode_dict_qcap['input_ids']
        input_ids_qcap = paddle.to_tensor(input_ids_qcap)
        qcap_feature, pooled_output= self.ernie(input_ids_qcap) #(b,length,dim)
        if self.mode == 'text':
            logits = self.classifier(qcap_feature[:,0,:].squeeze(1))
            return logits
        caps_feature = []
        for i,caption in enumerate (caps):
            encode_dict_cap = self.tokenizer(text = caption ,max_length = 128 ,truncation=True, padding='max_length')
            input_ids_caps = encode_dict_cap['input_ids']
            input_ids_caps = paddle.to_tensor(input_ids_caps)
            cap_feature, pooled_output= self.ernie(input_ids_caps) #(b,length,dim)
            caps_feature.append(cap_feature)
        caps_feature = paddle.stack(caps_feature,axis=0) #(b,num,length,dim)
        caps_feature = caps_feature.mean(axis=1)#(b,length,dim)
        caps_feature_fusion = self.attention_text(qcap_feature,caps_feature,caps_feature) #(b,length,dim)
        # =======================================================================
        if True:#self.ocr:
            img_text_feature = []
            for i,img_caption in enumerate (img_text):
                encode_dict_cap_img = self.tokenizer(text = img_caption ,max_length = 128 ,truncation=True, padding='max_length')
                input_ids_caps_img = encode_dict_cap_img['input_ids']
                input_ids_caps_img = paddle.to_tensor(input_ids_caps_img)
                cap_img_feature, pooled_img_output= self.ernie(input_ids_caps_img) #(b,length,dim)
                img_text_feature.append(cap_img_feature)
            img_text_feature = paddle.stack(img_text_feature,axis=0) #(b,num,length,dim)
            img_text_feature = img_text_feature.mean(axis=1)#(b,length,dim)
            img_text_feature_qcap = self.attention_text(qcap_feature,img_text_feature,img_text_feature) #(b,length,dim)
            img_text_feature_caps = self.attention_text(caps_feature,img_text_feature,img_text_feature) #(b,length,dim)
        # =======================================================================
        imgs_features = []
        for img in imgs:
            imgs_feature = self.resnet(img) #(length,dim)
            imgs_features.append(imgs_feature)
        imgs_features = paddle.stack(imgs_features,axis=0) #(b,length,dim)
        qImg_features = []
        for qImage in qImg:
            qImg_feature = self.resnet(qImage.unsqueeze(axis=0)) #(1,dim)
            qImg_features.append(qImg_feature)
        qImg_feature = paddle.stack(qImg_features,axis=0) #(b,1,dim)
        imgs_features = self.attention_image(qImg_feature,imgs_features,imgs_features) #(b,1,dim)
        # 计算用的是融合后的图像信息，而不是imgs_feature,可能会有问题？
        # 图像和文本证据融合
        imgs_features_qcap = (self.mid_attention_image_text(imgs_features)).reshape([qcap_feature.shape[0],qcap_feature.shape[1],qcap_feature.shape[2]])
        imgs_text_features_fusion_qcap = self.attention_image_text(imgs_features_qcap,imgs_features_qcap,qcap_feature)
        # 图像和文本融合
        imgs_features_caps = (self.mid_attention_image_text(imgs_features)).reshape([caps_feature.shape[0],caps_feature.shape[1],caps_feature.shape[2]])
        imgs_text_features_fusion_caps = self.attention_image_text(imgs_features_caps,imgs_features_caps,caps_feature)
        
        if self.dct:
            # ==================================
            dct_feature_batch = []
            for dct_img_ in dct_img:
                dct_feature_ = process_dct_img(dct_img_)
                dct_feature_ = self.dct_feature_model(dct_feature_)
                dct_feature_batch.append(dct_feature_)

            dct_feature = paddle.stack(dct_feature_batch,axis=0) #(b,num,length,dim)
            dct_feature = dct_feature.mean(axis=1)#(b,length,dim)   
            feature = paddle.concat(x=[caps_feature_fusion[:,0,:], imgs_text_features_fusion[:,0,:],imgs_features.squeeze(1),img_text_feature_qcap[:,0,:],img_text_feature_caps[:,0,:],dct_feature], axis=-1) 
        if self.ocr:
            feature = paddle.concat(x=[caps_feature_fusion[:,0,:], imgs_text_features_fusion[:,0,:],imgs_features.squeeze(1),img_text_feature_qcap[:,0,:],img_text_feature_caps[:,0,:]], axis=-1) 
        #feature = paddle.concat(x=[qcap_feature[:,0,:], caps_feature[:,0,:], img_text_feature[:,0,:],qImg_feature.squeeze(1), imgs_features.squeeze(1),imgs_text_features[:,0,:]], axis=-1) 
        # logits = self.classifier1(feature)
        # logits = self.classifier2(logits)
        def fusion():
            # 早期融合
            feature1 = paddle.concat(x=[qcap_feature[:,0,:], caps_feature[:,0,:],imgs_features[:,0,:],qImg_feature[:,0,:],img_text_feature[:,0,:]], axis=-1) 
            logits1 = self.mlp1(feature1)
            # 交叉融合
            fusion1,fusion2,fusion3,fusion4,fusion5,fusion6 = caps_feature_fusion,imgs_text_features_fusion_qcap,imgs_features,imgs_text_features_fusion_caps,img_text_feature_qcap,img_text_feature_caps
            if True:
                fusion3 = self.fusion_mlp(fusion3)
                fusion3 = paddle.tile(fusion3 , repeat_times=[128, 1])
                
                x1 = self.fusion_attention1(fusion1,fusion2,fusion2) + fusion1
                x2 = self.fusion_attention1(fusion2,fusion3,fusion3) + fusion2
                x3 = self.fusion_attention1(fusion3,fusion4,fusion4) + fusion3
                x4 = self.fusion_attention1(fusion4,fusion5,fusion5) + fusion4
                x5 = self.fusion_attention1(fusion5,fusion6,fusion6) + fusion6
                x2_1 = self.fusion_attention2(x1,x2,x2) + x1
                x2_2 = self.fusion_attention2(x2,x3,x3) + x2
                x2_3 = self.fusion_attention2(x3,x4,x4) + x3
                x2_4 = self.fusion_attention2(x4,x5,x5) + x4
                x3_1 = self.fusion_attention3(x2_1,x2_2,x2_2) + x2_1
                x3_2 = self.fusion_attention3(x2_2,x2_3,x2_3) + x2_2
                x3_3 = self.fusion_attention3(x2_3,x2_4,x2_4) + x2_3
                x4_1 = self.fusion_attention4(x3_1,x3_2,x3_2) + x3_1
                x4_2 = self.fusion_attention4(x3_2,x3_3,x3_3) + x3_2
                x5_1 = self.fusion_attention4(x4_1,x4_2,x4_2) + x4_1
                logits2 = self.fusion_mlp2(x5_1[:,0,:])
            else:
                feature2 = paddle.concat(x=[fusion1,fusion2,fusion3.squeeze(1),fusion4,fusion5,fusion6], axis=-1) 
                logits2 = self.mlp2(feature2)
            # 5个模态的特征，经过全连接层，进行晚期融合
            muti1 = self.mlp3(qcap_feature[:,0,:])
            muti2 = self.mlp3(caps_feature[:,0,:])
            muti3 = self.mlp4(imgs_features[:,0,:])
            muti4 = self.mlp4(qImg_feature[:,0,:])
            muti5 = self.mlp3(img_text_feature[:,0,:])
            logits3 =  muti1 + muti2 + muti3 + muti4 + muti5
            # 三种方式进行融合，还不知系数，因此先都为1

            logits = logits2+logits1 + logits3
            return logits

        
        if self.fusion_type:
            logits = fusion()
        else:
            # logits = self.classifier1(feature)
            # logits = self.classifier2(logits)
            logits = F.relu(self.classifier1(feature))
            logits = self.dropout(logits)
            #logits = self.classifier1(feature)
            #output = self.bn_1(output)
            logits = self.classifier2(logits)
        return logits

In [12]:
# 声明模型
model = NetWork("image")
print(model)

[2023-10-20 23:55:49,776] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-m-base/ernie_m_base.pdparams
W1020 23:55:49.781824 17873 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W1020 23:55:49.786636 17873 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
[2023-10-20 23:55:55,484] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-m-base/ernie_m.vocab.txt
[2023-10-20 23:55:55,487] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/ernie-m-base/ernie_m.sentencepiece.bpe.model
[2023-10-20 23:55:56,377] [    INFO] - tokenizer config file saved in /home/aistudio/.paddlenlp/models/ernie-m-base/tokenizer_config.json
[2023-10-20 23:55:56,380] [    INFO] - Special tokens file saved in /home/aistudio/.paddlenlp/models/ernie-m-base/special_tokens_map.json


NetWork(
  (ernie): ErnieMModel(
    (embeddings): ErnieMEmbeddings(
      (word_embeddings): Embedding(250002, 768, sparse=False)
      (position_embeddings): Embedding(514, 768, sparse=False)
      (layer_norm): LayerNorm(normalized_shape=[768], epsilon=1e-05)
      (dropout): Dropout(p=0.1, axis=None, mode=upscale_in_train)
    )
    (encoder): TransformerEncoder(
      (layers): LayerList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiHeadAttention(
            (q_proj): Linear(in_features=768, out_features=768, dtype=float32)
            (k_proj): Linear(in_features=768, out_features=768, dtype=float32)
            (v_proj): Linear(in_features=768, out_features=768, dtype=float32)
            (out_proj): Linear(in_features=768, out_features=768, dtype=float32)
          )
          (linear1): Linear(in_features=768, out_features=3072, dtype=float32)
          (dropout): Dropout(p=0, axis=None, mode=upscale_in_train)
          (linear2): Linear(in_features=3072,

# 六、训练配置

In [13]:
epochs = 4  
num_training_steps = len(train_dataloader) * epochs
warmup_steps = int(num_training_steps*0.1)
warmup_steps = 1118# 1118  # len(train_dataloader) * 2 * 0.1
print(num_training_steps,warmup_steps)
# 定义 learning_rate_scheduler，负责在训练过程中对 lr 进行调度
lr_scheduler = LinearDecayWithWarmup(1e-5, num_training_steps, warmup_steps)
# 训练结束后，存储模型参数
save_dir ="checkpoint/"
best_dir = "best_model"
# 创建保存的文件夹
os.makedirs(save_dir,exist_ok=True)
os.makedirs(best_dir,exist_ok=True)

decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

# 定义 Optimizer
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=1.2e-4,
    apply_decay_param_fun=lambda x: x in decay_params)

# 交叉熵损失
criterion = paddle.nn.loss.CrossEntropyLoss()

# 评估的时候采用准确率指标
metric = paddle.metric.Accuracy()

22368 1118


In [14]:
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    precision_recall_fscore_support,
)
# 自定义MultiLabelReport评价指标
class MultiLabelReport(Metric):
    """
    AUC and F1 Score for multi-label text classification task.
    """
    def __init__(self, name='MultiLabelReport', average='micro'):
        super(MultiLabelReport, self).__init__()
        self.average = average
        self._name = name
        self.reset()

    def reset(self):
        """
        Resets all of the metric state.
        """
        self.y_prob = None
        self.y_true = None

    def update(self, probs, labels):
        if self.y_prob is not None:
            self.y_prob = np.append(self.y_prob, probs.numpy(), axis=0)
        else:
            self.y_prob = probs.numpy()
        if self.y_true is not None:
            self.y_true = np.append(self.y_true, labels.numpy(), axis=0)
        else:
            self.y_true = labels.numpy()

    def accumulate(self):
        metrics = {}
        metrics["accuracy"] = accuracy_score(y_true=self.y_true, y_pred=self.y_prob)
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true=self.y_true, y_pred=self.y_prob,average="macro")
        average="macro"
        metrics[f"{average}_precision"] = precision
        metrics[f"{average}_recall"] = recall
        metrics[f"{average}_f1"] = f1
        return metrics



    def name(self):
        """
        Returns metric name
        """
        return self._name
metric2 = MultiLabelReport()

# 七、模型训练

In [15]:
# 定义线下评估 评价指标为acc 线上评估是f1score
@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader,epoch):
    gc.collect()
    model.eval()
    metric.reset()
    metric2.reset()
    losses = []
    kkk = 1 
    for batch in data_loader:    
        print(kkk)  
        kkk += 1
        labels, cap_batch, img_batch, qCap_batch, qImg_batch,img_text_batch,dct_img_batch = batch
        logits = model(qCap=qCap_batch,qImg=qImg_batch,caps=cap_batch,imgs=img_batch,img_text=img_text_batch,dct_img = dct_img_batch)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        correct = metric.compute(logits, labels)
        metric.update(correct)
        accu = metric.accumulate()
        probs = F.softmax(logits)
        metric2.update(labels,paddle.argmax(probs, axis=-1).reshape([-1,1]))
    eval_f1_score = metric2.accumulate()
    print("eval loss: %.5f, accu: %.5f" % (np.mean(losses), accu))
    model.train()
    metric.reset()
    metric2.reset()
    return np.mean(losses), accu,eval_f1_score

# 数据增强  
 https://zhuanlan.zhihu.com/p/546801916  
![](https://ai-studio-static-online.cdn.bcebos.com/96824b62c24a4fa6b60ed70658059a172d5c23b2c6d942f0b3b2e03b5e4a193d)

In [16]:
def mixgen(image,text,M,gama):
    for i in range(M):
        # image mixup
        image[i:] = (gama*image[i:]+(1-gama)*image[i+M])
        # text concatention
        text[i] = text[i] + text[i+M]
        return image,text
# print(cap_batch)
# img_batch,cap_batch = mixgen(img_batch,cap_batch,int(img_batch.shape[0]/2),0.5)
# print(cap_batch)

In [ ]:
# 定义训练
# param_dict = paddle.load('checkpoint/v3_epoch4_0.84115/model_state.pdparams')
# model.load_dict(param_dict)
# del param_dict
def do_train(model, criterion, metric, val_dataloader,train_dataloader):
    print("train run start")
    global_step = 0
    tic_train = time.time()
    best_accuracy=0.0
    # 梯度累计步
    gradient_accumulation_step = 2
    macro_f1 = []
    train_loss = []
    train_acc = []
    for epoch in range(1, epochs + 1):
        for step, batch in enumerate(train_dataloader, start=1): 
            #paddle.device.cuda.empty_cache()
            gc.collect()
            labels, cap_batch, img_batch, qCap_batch, qImg_batch,img_text_batch,dct_img_batch = batch
            if False:#random.random() <0.5: # 用于控制增强
                img_batch,cap_batch = mixgen(img_batch,cap_batch,int(img_batch.shape[0]/2),0.5)  
                # 为了应对batch中末尾的text没有拼接，给没有增强的加上空值，使其每个batch里的text维度一样
                num_max = max([len(i) for i in cap_batch]) 
                num_max = 50 if num_max>50 else num_max
                cap_batch = [j+[""]*(num_max-len(j)) if len(j) < num_max else j[:num_max] for j in cap_batch]
            probs = model(qCap=qCap_batch,qImg=qImg_batch,caps=cap_batch,imgs=img_batch,img_text=img_text_batch,dct_img = dct_img_batch)
            loss = criterion(probs, labels)/gradient_accumulation_step
            correct = metric.compute(probs, labels)
            metric.update(correct)
            acc = metric.accumulate()
            # 输出类别的概率
            probs = F.softmax(probs)
            metric2.update(labels,paddle.argmax(probs, axis=-1).reshape([-1,1]))
            f1_score = metric2.accumulate()
            macro_f1.append(round(f1_score["macro_f1"],4))
            train_loss.append(round(loss.item(),4))
            train_acc.append(round(acc,4))
            # 画指标
            if global_step % 10 == 0:
                plt.figure(figsize=(12, 8))
                plt.subplot(3, 1, 1)
                plt.plot(macro_f1,color='r')
                plt.ylabel('val/ macro_f1')
                plt.title('macro_f1')
                plt.subplot(3, 1, 2)
                plt.plot(train_loss,color='b')
                plt.ylabel('train_loss')
                plt.title('train_loss')
                plt.subplot(3, 1, 3)
                plt.plot(train_acc,color='g')
                plt.ylabel('train_acc')
                plt.xlabel('step')
                plt.title('train_acc')
                plt.savefig(f'train_acc.png')
            global_step += 1 
            # 每间隔 100 step 输出训练指标
            if global_step % 100 == 0:
                print(
                    "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, f1_score: %.5f, speed: %.2f step/s"
                    % (global_step, epoch, step, loss, acc,f1_score['macro_f1'],
                        10 / (time.time() - tic_train)))
                tic_train = time.time()
            loss.backward()
           
            if (step+1) % gradient_accumulation_step == 0:  
                optimizer.step()   # 更新参数                                       
                lr_scheduler.step()
                optimizer.clear_grad() #梯度清零
                paddle.device.cuda.empty_cache()
            # 保存最优模型
            if global_step  == 16876:
                save_param_path = os.path.join("model", 'model_state.pdparams')
                print(save_param_path)
                paddle.save(model.state_dict(), save_param_path)
            # 每间隔一个epoch 在验证集进行评估
            if global_step % len(train_dataloader) == 0:
                eval_loss,eval_accu,eval_f1_score=evaluate(model, criterion, metric, val_dataloader,epoch)
                # save_param_path = os.path.join(save_dir+str(epoch), 'model_state.pdparams')
                # paddle.save(model.state_dict(), save_param_path)
                if(best_accuracy<eval_accu):
                    best_accuracy=eval_accu
                    # 保存模型
                    #save_param_path = os.path.join(best_dir, 'model_best.pdparams')
                    paddle.save(model.state_dict(), save_param_path)
do_train(model, criterion, metric, val_dataloader,train_dataloader) 

train run start


In [ ]:
# 压缩
#zip -r submission.zip  ernie-m-base  model  PaddleOCR-2.6.0  paddlenlp paddlenlp-2.4.2.dist-info predict.py

In [1]:
!ls -l

总用量 3375848
drwxr-xr-x  3 aistudio aistudio       4096 10月 20 21:41 ernie-m-base
-rw-r--r--  1 aistudio aistudio     148346 10月 20 23:58 main_warmup2_增强_ocr_频域特征v2_融合_epoch4_100_0.84122-final.ipynb
drwxr-xr-x  3 aistudio aistudio       4096 10月 20 21:41 model
drwxr-xr-x 18 aistudio aistudio       4096 10月 20 21:41 paddlenlp
drwxr-xr-x  2 aistudio aistudio       4096 10月 20 21:41 paddlenlp-2.4.2.dist-info
drwxr-xr-x 19 aistudio aistudio       4096 10月 20 23:01 PaddleOCR-2.6.0
-rw-r--r--  1 aistudio aistudio      33819 10月 20 22:12 predict.py
-rw-r--r--  1 aistudio aistudio       8393 10月 20 23:08 predict_system.py
-rw-r--r--  1 aistudio aistudio       3015 10月 20 23:35 README.md
-rw-r--r--  1 aistudio aistudio        282 10月 20 23:50 requirements.txt
-rw-r--r--  1 aistudio aistudio         98 10月 20 22:04 result.csv
-rw-r--r--  1 aistudio aistudio 3456548582 10月 20 22:16 submission.zip
-rw-r--r--  1 aistudio aistudio      74681 10月 20 23:57 train_acc.png
-rw-r--r--  1 aistudio aistudio 

In [ ]:
!zip -r 傻瓜oop815的团队.zip  ernie-m-base main_warmup2_增强_ocr_频域特征v2_融合_epoch4_100_0.84122-final.ipynb  model    paddlenlp paddlenlp-2.4.2.dist-info predict.py PaddleOCR-2.6.0  predict.py   predict_system.py  README.md   requirements.txt result.csv submission.zip  团队信息.txt

  adding: ernie-m-base/ (stored 0%)
  adding: ernie-m-base/model_config.json (deflated 43%)
  adding: ernie-m-base/tokenizer_config.json (deflated 34%)
  adding: ernie-m-base/special_tokens_map.json (deflated 40%)
  adding: ernie-m-base/vocab.txt (deflated 47%)
  adding: ernie-m-base/ernie_m.sentencepiece.bpe.model (deflated 49%)
  adding: ernie-m-base/.ipynb_checkpoints/ (stored 0%)
  adding: ernie-m-base/.ipynb_checkpoints/model_config-checkpoint.json (deflated 43%)
  adding: ernie-m-base/.ipynb_checkpoints/vocab-checkpoint.txt (deflated 47%)
  adding: ernie-m-base/.ipynb_checkpoints/ernie_m.vocab-checkpoint.txt (deflated 47%)
  adding: ernie-m-base/ernie_m.vocab.txt (deflated 47%)
  adding: ernie-m-base/model_state.pdparams